In [584]:
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import datetime
import winsound
from sklearn import linear_model
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

In [585]:
df = pd.read_csv('finalCleaning.gzip',compression='gzip',parse_dates=[1],infer_datetime_format=True, index_col=0)

winsound.Beep(2000,300) # Beep when code is executed

C:\Users\Matthias\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [586]:
def parseAsTime(row):
    row.departure_schedule = datetime.datetime.strptime(row.departure_schedule,"%H:%M:%S").time()
    row.departure_actual = datetime.datetime.strptime(row.departure_actual,"%H:%M:%S").time()
    row.arrival_schedule = datetime.datetime.strptime(row.arrival_schedule,"%H:%M:%S").time()
    row.arrival_actual = datetime.datetime.strptime(row.arrival_actual,"%H:%M:%S").time()
    return row

In [587]:
#df = df.apply(parseAsTime,axis=1) too long

#winsound.Beep(2000,300) # Beep when code is executed

In [588]:
df_ml = df.loc[0:9999,["airline_code","departure_lat","departure_lon","arrival_lat","arrival_lon","departure_schedule","arrival_schedule","departure_delay"]]

In [589]:
def parseAsInt(row):
    row.departure_schedule = row.departure_schedule.minute + (100 * row.departure_schedule.hour)
    row.arrival_schedule = row.arrival_schedule.minute + (100 * row.arrival_schedule.hour)
    return row

In [590]:
df_ml = df_ml.apply(parseAsInt, axis=1)

AttributeError: ("'str' object has no attribute 'minute'", 'occurred at index 0')

In [ ]:
def getScatterPlot(indexOfPredictor):
    colname = df_ml.columns[indexOfPredictor]
    print("--------------------"+colname+"--------------------")
    x = df_ml.drop("departure_delay",axis=1).values
    y = df_ml.departure_delay.values
    reg = linear_model.LinearRegression()
    x_chosenPredictor = x[:,indexOfPredictor].reshape(-1,1)
    reg.fit(x_chosenPredictor,y)
    plt.scatter(x_chosenPredictor, y)
    plt.ylabel('delay')
    plt.xlabel(colname)
    plt.show()
    prediction_space = np.linspace(min(x_chosenPredictor),max(x_chosenPredictor)).reshape(-1,1)
    plt.scatter(x_chosenPredictor, y, color='blue')
    plt.plot(prediction_space, reg.predict(prediction_space),color='black',linewidth=3)
    plt.show()

In [ ]:
def doRidge(alphaParam):
    print("--------------------RIDGE--------------------")
    print(alphaParam)
    x = df_ml.drop("departure_delay",axis=1).values
    y = df_ml.departure_delay.values
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3, random_state = 1)
    ridge = Ridge(alpha=alphaParam, normalize=True)
    ridge.fit(x_train, y_train)
    ridge_pred = ridge.predict(x_test)
    return ridge.score(x_test, y_test)

In [ ]:
def doLasso(alphaParam):
    print("--------------------LASSO--------------------")
    print(alphaParam)
    x = df_ml.drop("departure_delay",axis=1).values
    y = df_ml.departure_delay.values
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3, random_state = 1)
    lasso = Lasso(alpha=alphaParam, normalize=True)
    lasso.fit(x_train, y_train)
    lasso_pred = lasso.predict(x_test)
    return lasso.score(x_test, y_test)

In [ ]:
for i in range(0,7):
    getScatterPlot(i)

In [ ]:
for i in (-0.5,-0.4,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.4,0.5):
    print(doRidge(i))

In [ ]:
print(doLasso(0.1))